# 'Applied Data Science Capstone' Project

This notebook will mainly be used for the above-mentioned capstone project.

In [264]:
# -------------------
# IMPORTING LIBRARIES
# -------------------

import pandas as pd
import numpy as np

import requests  # library to handle requests
from bs4 import BeautifulSoup as soup  # to scrape the wikipedia data

!pip install geocoder
import geocoder  # to get latitude/longitudes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

import json  # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors

from sklearn.cluster import KMeans  # import k-means from clustering stage

!pip install folium
import folium  # map rendering library

     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01


In [13]:
# Print the following the statement: Hello Capstone Project Course!
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


---

## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

1. Start by creating a new Notebook for this assignment.

In [16]:
# You are reading it right now, so this obviously already happened.

---

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![alt text](https://bit.ly/3pF2zYc "Screenshot of pandas dataframe")


3. To create the above dataframe:
  + The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
  + Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
  + More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11**  in the above table.
  + If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.
  + Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
  + In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.

In [253]:
# ----------------
# SCRAPING OF DATA
# ----------------

# define url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# request
r = requests.Session()
response = r.get(url, timeout=10)  # expected value: 200

# using BeautifulSoup package for scraping webpages
soup = BeautifulSoup(response.content, 'html.parser')
pretty_soup = soup.prettify()

# find the table in the html
table=soup.find('table')

# Number of columns in the table
for row in right_table.findAll("tr"):
    columns = row.findAll("td")
len(columns)

# Number of rows in the table including header
rows = right_table.findAll("tr")
len(rows)

print("The table has a size of " + str(len(columns)), "columns and " + str(len(rows)-1), "rows (not including the header).")

The table has a size of 3 columns and 180 rows (not including the header).


In [254]:
# ----------------------
# CREATING THE DATAFRAME
# ----------------------

# getting column names / header
header = [th.text.rstrip() for th in rows[0].find_all("th")]  # optional: print(header)

# getting first row information as a test
row1 = [td.text.rstrip() for td in rows[1].find_all("td")]  # optional: print(row1)

# define empty list to fill with row information
allrows = []

# getting all rows now
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all('td')]
    allrows.append(data)  # optional: print(allrows)

# creating the DataFrame
df = pd.DataFrame(allrows)
df.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [255]:
# -------------------
# DATA PRE-PROCESSING
# -------------------

# assigning header to dataframe
df.columns = header

# deleting cells with a borough that is "Not assigned".
df.replace('Not assigned', np.NaN, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)

# setting Neighbourhoods that are "Not assigned" to the same value as the Borough
df["Neighbourhood"].replace(np.NaN, "Borough", inplace=True)

# resetting the index
df.reset_index(drop=True, inplace=True)

# having a glimpse
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [205]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print("df.shape =", df.shape,
      "--> so that means our DataFrame \"df\" consists of", df.shape[0], "rows and", df.shape[1], "columns.")

df.shape = (103, 3) --> so that means our DataFrame "df" consists of 103 rows and 3 columns.


---

4. Submit a link to your Notebook on your Github repository. **(10 marks)**

In [211]:
## submit a link to the new Notebook on your Github repository
print("https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb")

https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb


*Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas  to read the table into a pandas dataframe.*

*Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/*

*Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.*

---

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking  postal code **M5G** as an example, your code would look something like this:

```
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

![alt text](https://bit.ly/2ZFUGXI "Screenshot of pandas dataframe")

**Important Note:** There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

In [252]:
# ----------------------------------------------------
# GETTING LATITUDE & LONGITUDE FOR EVERY NEIGHBOURHOOD
# ----------------------------------------------------

# downloading the provided csv file
!wget -q -O "geospatial_data.csv" http://cocl.us/Geospatial_data

In [258]:
# reading csv into a DataFrame
df_geo = pd.read_csv("geospatial_data.csv")

# sorting both DataFrames by Postal Code
df_geo.sort_values(by=["Postal Code"], inplace=True)
df.sort_values(by=["Postal Code"], inplace=True)

# combining the two DataFrames into one
df_new = pd.merge(df, df_geo, on="Postal Code")
df_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. **(2 marks)**

**Note:** While including the link do not copy paste the URL. Use the embedded link option in the formatting  tools of the Response field to include the link. Check the  displayed in image below

In [259]:
## submit a link to the new Notebook on your Github repository
print("https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb")

https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb


---

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2. to generate maps to visualize your neighborhoods and how they cluster together.

In [268]:
# getting the coordinates of Toronto
address = 'Toronto, OR'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.7453749, -79.44838361923703.


In [272]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Indeed, let us only work with boroughs that contain the word **Toronto**:

In [439]:
# creating a DataFrame only with Boroughs that contain the word "Toronto"
toronto_data = df_new[df_new['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let's focus the map on Central Toronto now:

In [294]:
address = 'Central Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Central Toronto are 43.6534817, -79.3839347.


In [295]:
# create map of Manhattan using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

Utilizing the Foursquare API to explore the neighbourhoods and segment them now.

In [340]:
CLIENT_ID = '5RUVSMYVTJQQCZOXDKXNBJ5R4EAI02SUKLPNDKPCNF4CNPV1'
CLIENT_SECRET = '1SOC0THNTI0EUQ5FHLMXSDSG4LXTJPJT144F31VMWTNCXF12'
ACCESS_TOKEN = 'GVUNLMHFLCC5P4W0FJXUVDM5ZRGWNO2ZF0PWIQBUCR3CZ0XE'
VERSION = '20210222' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

## 1) I'd like to have a closer look at Cabbagetown, since I really like all sorts of Kraut:

In [326]:
cabbage = toronto_data[toronto_data['Neighbourhood'].str.contains("Cabbage")].reset_index(drop=True)
cabbage

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675


In [327]:
neighbourhood_latitude = toronto_data.loc[11, 'Latitude']    # neighbourhood latitude value
neighbourhood_longitude = toronto_data.loc[11, 'Longitude']  # neighbourhood longitude value
neighbourhood_name = toronto_data.loc[11, 'Neighbourhood']   # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude)
     )

Latitude and longitude values of St. James Town, Cabbagetown are 43.667967, -79.3676753.


Let's get the top 50 venues that are in Cabbagetown within a radius of 500 meters:

In [349]:
RADIUS = 500 # define radius

url_4 = "https://api.foursquare.com/v2/venues/explore?client_id=5RUVSMYVTJQQCZOXDKXNBJ5R4EAI02SUKLPNDKPCNF4CNPV1\
        &client_secret=1SOC0THNTI0EUQ5FHLMXSDSG4LXTJPJT144F31VMWTNCXF12\
        &ll=43.667967,-79.3676753\
        &auth_token=GVUNLMHFLCC5P4W0FJXUVDM5ZRGWNO2ZF0PWIQBUCR3CZ0XE\
        &v=20210222\
        &radius=500\
        &limit=50"
results = requests.get(url_4).json()
results

{'meta': {'code': 200, 'requestId': '60353396474da31928ae6434'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Cabbagetown',
  'headerFullLocation': 'Cabbagetown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6724670045, 'lng': -79.3614658826597},
   'sw': {'lat': 43.663466995499995, 'lng': -79.3738847173403}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b646a6ff964a5205cb12ae3',
       'name': 'Cranberries',
       'location': {'address': '601 Parliament St.',
        'crossStreet': 'at Wellesley St. E',
        'lat': 43.6678427705951,
        'lng': -79.36940687874281,
        'labeledLatLngs': [{'lab

Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [350]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the **json** and structure it into a pandas dataframe.

In [353]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)  # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Cranberries,Diner,43.667843,-79.369407
1,Murgatroid,Restaurant,43.667381,-79.369311
2,F'Amelia,Italian Restaurant,43.667536,-79.368613
3,Merryberry Cafe + Bistro,Café,43.666630,-79.368792
4,Cabbagetown Brew,Café,43.666923,-79.369289


In [352]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


## 2) Explore all Neighbourhoods featuring "Toronto" in their name

Let's create a function to repeat the same process to all the neighbourhoods featuring "Toronto":

In [371]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe called cabbage_venues:

In [372]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [375]:
# size of resulting dataframe
print("The size of the resulting dataframe is", toronto_venues.shape)

# let's group by neighbourhoods
toronto_venues.groupby('Neighbourhood').count()

# unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

toronto_venues

The size of the resulting dataframe is (1167, 7)
There are 219 uniques categories.


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
...,...,...,...,...,...,...,...
1162,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1163,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
1164,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
1165,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


## Analyse each neighbourhood
Deeper Analysis of Toronto-Neighbourhoods via One-Hot-Encoding:

In [376]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [377]:
# grouping rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.00000,0.000000,0.00000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000
4,Central Bay Street,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.00000,0.000000,0.02000,0.020000
5,Christie,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000
6,Church and Wellesley,0.02,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.02,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.040000
7,"Commerce Court, Victoria Hotel",0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.00000,0.000000,0.00000,0.000000
8,Davisville,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.030303,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000
9,Davisville North,0.00,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00000,0.000000


Let's print each neighborhood along with the top 5 most common venues:

In [379]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.06
2      Restaurant  0.04
3  Farmers Market  0.04
4     Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3       Nightclub  0.05
4    Intersection  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0      Farmers Market  0.06
1  Light Rail Station  0.06
2       Burrito Place  0.06
3          Restaurant  0.06
4              Garden  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.18
1             Café  0.

In [383]:
# turning that information into a dataframe

# first defining a function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# then applying it
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Farmers Market,Cheese Shop,Pharmacy,Beer Bar,Comfort Food Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Nightclub,Stadium,Intersection,Bakery,Italian Restaurant,Restaurant,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Farmers Market,Butcher,Auto Workshop,Spa,Burrito Place,Recording Studio,Smoke Shop,Fast Food Restaurant,Skate Park,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Bar,Coffee Shop,Sculpture Garden
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Bubble Tea Shop,Burger Joint,Italian Restaurant,Middle Eastern Restaurant,Salad Place,Ramen Restaurant,Portuguese Restaurant


## 4) Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [441]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [442]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [443]:
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Neighborhood,Trail,Pub,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Bakery,Pub,Pizza Place
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Pet Store,Pub,Brewery,Sandwich Place,Italian Restaurant,Restaurant,Steakhouse,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Gastropub,Brewery,Café,Bakery,American Restaurant,Neighborhood,Cheese Shop,Clothing Store,Gay Bar
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Finally, let's visualize the resulting clusters:

In [444]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5) Examine Clusters

Examining each cluster and determining the discriminating venue categories that distinguish each cluster.
Based on the defining categories, we can then assign a name to each cluster.

In [402]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Asian Restaurant,Health Food Store,Neighborhood,Trail,Pub,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Bakery,Pub,Pizza Place
2,East Toronto,0,Park,Fast Food Restaurant,Pet Store,Pub,Brewery,Sandwich Place,Italian Restaurant,Restaurant,Steakhouse,Ice Cream Shop
3,East Toronto,0,Coffee Shop,Gastropub,Brewery,Café,Bakery,American Restaurant,Neighborhood,Cheese Shop,Clothing Store,Gay Bar
4,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
5,Central Toronto,0,Pizza Place,Department Store,Park,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Hotel,Food & Drink Shop,Dumpling Restaurant,Donut Shop
6,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Café,Seafood Restaurant,Spa,Diner,Sporting Goods Shop,Salon / Barbershop,Fast Food Restaurant
7,Central Toronto,0,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Pizza Place,Gym,Sushi Restaurant,Toy / Game Store,Gas Station
9,Central Toronto,0,Coffee Shop,Pub,Fried Chicken Joint,Liquor Store,Sandwich Place,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Bakery,Italian Restaurant,Pub,Market,Pizza Place,Pet Store,Sandwich Place


In [403]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Trail,Tennis Court,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [404]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Home Service,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [405]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [406]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


---

## **The inner city of Toronto seems to be a rather homogeneous area, with many neighbourhoods being clustered in the first cluster, showing a strong focus on coffee shops, cafés, and bars. The other four clusters seem to be more focused on parks, playgrounds, and gardens.**

---

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. **(3 marks)**

In [260]:
## submit a link to the new Notebook on your Github repository
print("https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb")

https://github.com/ZeLilFish/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb


# Thanks for reading my Notebook!
## Jens